In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
Nodes = 

In [ ]:
# константы
E = 6e6 # H/cv^2 (модуль упругости (Юнга))
mu = 0.25
t = 2
p = 20
alpha = 7e-6
dT = 15

In [ ]:
# коэффициенты матрицы 
b = Nodes[[1,2, 0], 1] - Nodes[[2,0,1], 1]
b

In [ ]:
с = Nodes[[2, 0, 1], 0] - Nodes[[1, 2, 0], 0]
c

In [ ]:
# функция для вычисления площади элемента
def triarea(a, b, c):
    ab = b - a
    ac = c - a
    return 0.5 * (ab[0]*ac[1] - ab[1]*ac[0])

In [ ]:
A = triarea(Nodes[0], Nodes[1], Nodes[2])

In [ ]:
# матрица градиентов
B1 = np.array([b[0], 0, b[1], 0, b[2], 0])
B1 = np.array([0, c[0], 0, c[1], 0, c[2]])
B = np.vstack(
    (B1, B2, np.roll(B1, 1) + np.roll(B1, 1))/2
)

print(B*2*A)

In [ ]:
# матрица упругих постоянных
D = E / (1 - mu**2) * np.array([[ 1, mu, 0]
                                [mu,  1, 0]
                                [ 0,  0, 0.5*(1-mu)]])

print(D * (1 - mu**2) / E * 8)

In [ ]:
# матрица жесткости
K = np.matmul(np.matmul(B.T, D), B) * t * A
print(K / 133330)
print(B.T.dot(D@B) * t * A / 133330)

In [ ]:
# деформации, вызванные изменением температуры
eps0 = alpha * dT * np.array([[1,1,0]]).T
print(eps0 * 1e5)

In [ ]:
# узловые усилия из-за тумпературных деформаций
Fth = np.matmul(np.matmul(B.T, D), eps0) * t * A
Fth

In [ ]:
# длинна ребра jk
Ljk = np.linalg.norm(Nodes[1] - Nodes[2])
Ljk

In [ ]:
# косинус и синус угла, под которым ориентировано распределённое усилие p
cost = Nodes[2, 1] / Ljk
sint = (Nodes[1, 0] - Nodes[2, 0]) / Ljk
cost, sin

In [ ]:
# пересчет распеределенного усилия на узлы
Njk = np.array([[0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
                [0.0, 0.0, 0.0, 1.0, 0.0, 1.0]])

px = p * cost
py = p * sint
Fp = 0.5 * Ljk * t * np.matmul(Njk.T, np.array([[px, py]]).T)
Fp

In [ ]:
# сумарный вектор усилий 
F = Fth + Fp
F

In [ ]:
# внесение граничных условий в виде нулевых перемещений
# узел i закреплен полностью, узел j - только по вертикали
dof = [0, 1, 3]
K[dof, :] = 0
K[:, dof] = 0
K[dof, dof] = 1


In [ ]:
plt.spy(K)

In [ ]:
print(K)

In [ ]:
print(F)

In [ ]:
U = np.linalg.solve(K, F)
U * 1000

In [ ]:
U.reshape(-1,2) * 1000

In [ ]:
# результирующее положение узлов с учетом масштабного коэффициента
resNodes = Nodes + 1000 * U.reshape(-1, 2)

In [ ]:
plt.figure(figsize=(4,4))
plt.triplot(Nodes[:, 0], Nodes[:, 1], [[0, 1, 2]], color='b', alpha=0.5)
plt.triplot(resNodes[:, 0], resNodes[:, 1], [[0, 1, 2]], color='r')

In [ ]:
# расчитать тензор деформации 
#
#

In [2]:
# [ Sx Txy 0]
# [Txy  Sy 0]
# [  0   0 0]

In [ ]:
eps = B.dot(U)
sig = D.dot(eps) - D.dot(eps0)
sig

In [ ]:
s1 = 0.5 * (sig[0] + sig[1]) + np.sqrt((0.5 * (sig[0] - sig[1]))**2 + sig[2]**2)
s2 = 0.5 * (sig[0] + sig[1]) - np.sqrt((0.5 * (sig[0] - sig[1]))**2 + sig[2]**2)
s1[0], s2[0]